<a href="https://colab.research.google.com/github/A-Peoples/BCS-Recreated/blob/main/BCS_Recreated%201.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Packages**

**Get Your key here: https://collegefootballdata.com/key**

In [1]:
!pip install cfbd --upgrade

import numpy as np
import pandas as pd
import cfbd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = #put key here
configuration.api_key_prefix['Authorization'] = 'Bearer'

api_config = cfbd.ApiClient(configuration)

response = games = cfbd.GamesApi(api_config).get_games(year=2024, season_type='regular')

data = pd.DataFrame.from_records([g.to_dict() for g in games])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.8/358.8 kB 13.1 MB/s eta 0:00:00


**Importing Data and Team Names**

In [2]:
data = pd.DataFrame.from_records([g.to_dict() for g in games])


data = data[
    (pd.notna(data['home_conference']))
    & (pd.notna(data['away_conference']))
]

data.head()

data.columns.values

team_names = data['home_team'].unique()
final_data = pd.DataFrame()

**Creating Values**

In [3]:
data["margin"] = data["home_points"] - data["away_points"]

final_team = pd.DataFrame()
wins = 0
losses = 0
c_wins = 0
c_losses = 0
margin = 0
total_games = 0
conference = ''
for team_name in team_names:
  points_scored = 0
  points_against = 0
  eilo = 0
  data["margin"] = data["home_points"] - data["away_points"]
  data
  team = data.loc[(data['home_team'] == team_name) | (data['away_team'] == team_name)]
  team
  main_team = pd.DataFrame()
  for index, row in team.iterrows():
    if row["home_team"] == team_name:
      points_scored = points_scored + row['home_points']
      points_against = points_against + row['away_points']
      total_games +=1
      eilo = eilo + row["home_pregame_elo"]
      if row["home_points"] >  row["away_points"]:
        if row["conference_game"] == True:
          c_wins +=1
        wins +=1
      elif row["home_points"] <  row["away_points"]:
        if row["conference_game"] == True:
          c_losses +=1
        losses +=1
      margin += row["margin"]
      conference = row['home_conference']
    elif row["away_team"] == team_name:
      points_against = points_against + row['home_points']
      points_scored = points_scored + row['away_points']
      eilo = eilo + row["away_pregame_elo"]
      total_games +=1
      if row["home_points"] <  row["away_points"]:
        if row["conference_game"] == True:
          c_wins +=1
        wins +=1
      elif row["home_points"] >  row["away_points"]:
        if row["conference_game"] == True:
          c_losses +=1
        losses +=1
      margin += row["margin"]
      conference = row['away_conference']
      points_scored = points_scored / total_games
      points_against = points_against / total_games
  main_team = pd.DataFrame({
      "team": [team_name],
      "wins": [wins],
      "losses": [losses],
      "c_wins": [c_wins * 0.5],
      "c_losses": [c_losses * 1.5],
      'conference': [conference],
      'points_scored': [points_scored],
      'points_against': [points_against],
      "margin": [margin],
      'eilo': [eilo],
      })
  final_team = pd.concat([final_team, main_team], ignore_index=True)
  wins = 0
  losses = 0
  c_wins = 0
  c_losses = 0
  margin = 0
  total_games = 0

**Creating the Ranking System**

In [6]:
FBS = final_team.loc[(final_team['conference'] == "SEC") | (final_team['conference'] == "Sun Belt") |(final_team['conference'] == "Big Ten")
| (final_team['conference'] == "Pac-12") | (final_team['conference'] == 'Conference USA') | (final_team['conference'] == "FBS Independents") |
                     (final_team['conference'] == "ACC")| (final_team['conference'] == "Mountain West") | (final_team['conference'] == "Big 12") | (final_team['conference'] == "Mid American")
                     | (final_team['conference'] == "American Athletic")]
FBS = FBS.sort_values(by=['wins', "margin", "c_wins"], ascending=False)
FBS['eilo'] = FBS['eilo'] / 13
FBS["points"] = ((FBS['wins'] + FBS['c_wins']) - ((FBS['losses'] + FBS['c_losses']) * 10) + (FBS['eilo'] * 0.1)) * 0.1
FBS['points'] = FBS['points'].round(4)
FBS = FBS.sort_values(by=["points"], ascending=False)
FBS["rank"] = range(1, len(FBS) + 1)
FBS = FBS.head(25)
FBS = FBS[["rank","team","conference", "wins", "losses", "points"]]
FBS

,rank,team,conference,wins,losses,points
88,1,Texas,SEC,5,0,19.4669
94,2,Ohio State,Big Ten,5,0,19.4223
143,3,Oregon,Big Ten,5,0,18.6423
230,4,Penn State,Big Ten,5,0,18.0300
312,5,Notre Dame,FBS Independents,4,1,17.5385
324,6,Miami,ACC,6,0,16.9846
67,7,Georgia,SEC,4,1,16.8254
165,8,LSU,SEC,4,1,16.4900
125,9,Alabama,SEC,4,1,16.4638
98,10,Iowa State,Big 12,5,0,16.4492
